# Imports

In [1]:
import os
import sys

sys_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(sys_path)

In [2]:
from utils.code_pred_utils import get_top_n_predictions, accuracy_calculator
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from tqdm import tqdm

root_dir_path = os.getcwd()
root_dir_path = os.path.abspath(os.path.split(root_dir_path)[0])

In [3]:
model_save_path = rf'{root_dir_path}/models/'

test_df = pd.read_csv(rf'{root_dir_path}/data/test_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)

test_data = TabularDataset(test_df)
label = "kpb_activity_code"

# Activity-Code Prediction

In [4]:
predictor = TabularPredictor.load(model_save_path)

predictor.model_best

'WeightedEnsemble_L3_FULL'

In [ ]:
train_df = pd.read_csv(rf'{root_dir_path}/data/train_val_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)

train_data = TabularDataset(train_df)

importance_scores = predictor.feature_importance(train_data)
print(importance_scores)

In [5]:

single_predicted_code = predictor.predict(data = test_data, model = predictor.model_best)

test_df[label] = single_predicted_code

In [ ]:
# This cell is for lifting out a user defined amount of predictions to put into the result DataFrame, it is incredibly inefficent and is not a good solution.

predicted_codes_df = pd.DataFrame

for i in tqdm(range(len(test_data))):

    predicted_codes = predictor.predict_proba(data = test_data.iloc[[i]], model = predictor.model_best)
    predicted_codes_df = get_top_n_predictions(predicted_codes, 3)

In [9]:
predicted_codes = predictor.predict_proba(data = test_data, model = predictor.model_best)

predicted_codes.apply(func=lambda row: row.nlargest(3), axis=1)

,A_ADM_600_1,A_ADM_600_10,A_ADM_600_11,A_ADM_600_16,A_ADM_600_16B,A_ADM_600_20,A_ADM_600_24,A_ADM_600_25,A_ADM_600_3,A_ADM_600_5,...,A_ÖPP_904_0,A_ÖPP_904_1,A_ÖVR_500_0_1,A_ÖVR_500_0_10,A_ÖVR_500_0_16,A_ÖVR_500_0_4,A_ÖVR_500_0_7,A_ÖVR_504_0,A_ÖVR_504_1,A_ÖVR_505_0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.130435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.108696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.108696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181,0.086957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
test_df[label] = predicted_codes

ValueError: Cannot set a DataFrame with multiple columns to the single column kpb_activity_code

In [6]:
test_df.to_csv(rf'{root_dir_path}/predicted_data/predicted_data_csv_2.csv', sep= '|')

# Some extra testing

In [7]:
train_df = pd.read_csv(rf'{root_dir_path}/data/train_val_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)
train_df.drop(columns=["KPB AKTIVITETSKOD"])

train_data = TabularDataset(train_df)

In [8]:

single_predicted_code = predictor.predict(data = test_data, model = predictor.model_best)

train_df[label] = single_predicted_code

In [9]:
train_df.to_csv(rf'{root_dir_path}/predicted_data/predicted_train_data_csv.csv', sep= '|')

In [10]:
predictor.evaluate(train_data)

{'accuracy': 0.9849791919542223,
 'balanced_accuracy': 0.8746936356547245,
 'mcc': 0.9848439509203382}

In [11]:
eval_data = pd.read_csv(rf'{root_dir_path}/predicted_data/predicted_train_data_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)

total_truth, total_false, accuracy_score = accuracy_calculator(eval_data)

print(f"Total true predictions: {total_truth}")
print(f"Total false predictions: {total_false}")
print(f"Total predictions: {total_false + total_truth}")
print(f"Model factual accuracy = {accuracy_score}")

46136it [00:04, 9973.14it/s] 

Total true predictions: 383
Total false predictions: 45753
Total predictions: 46136
Model factual accuracy = 0.008301543263395179
